In [669]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/captcha-hacker/sample_submission.csv
/kaggle/input/captcha-hacker/test/task1/x4LPcV5n6IvLj4vz.png
/kaggle/input/captcha-hacker/test/task1/W88fVMlAs5IpsXn4.png
/kaggle/input/captcha-hacker/test/task1/ZWDL6pUMfPu5c9jh.png
...
/kaggle/input/captcha-hacker/test/task2/jMalnsI5a5IWxYAi.png
/kaggle/input/captcha-hacker/test/task2/ihE9HHgyOINGEMcO.png
/kaggle/input/captcha-hacker/test/task2/ZATEVW3P5s0akZjd.png
...
/kaggle/input/captcha-hacker/test/task3/cXBlxYfvQWbiK7dn.png
/kaggle/input/captcha-hacker/test/task3/5gEp1jR9jNNfuqlk.png
/kaggle/input/captcha-hacker/test/task3/hEQ0WQtB9B7j8C2f.png
...
/kaggle/input/captcha-hacker/train/annotations.csv
/kaggle/input/captcha-hacker/train/task1/H85RQ6dbWUvLSIDV.png
/kaggle/input/captcha-hacker/train/task1/n2GC8uY1N4QfvVxe.png
/kaggle/input/captcha-hacker/train/task1/XOqfRx2R6SnoEjFr.png
...
/kaggle/input/captcha-hacker/train/task2/Mr4B2zxXk92hyzn9.png
/kaggle/input/captcha-hacker/train/task2/SIuRCnlK8VS91FhX.png
/kaggle/input/captcha-h

In [670]:
import csv
import cv2
import numpy as np
import random
import tensorflow as tf
import os
import torchvision
import torchvision.transforms as transforms
from torchvision import io
from tqdm import tqdm
import torchvision.models as models
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [671]:
TRAIN_PATH = "/kaggle/input/captcha-hacker/train"
TEST_PATH = "/kaggle/input/captcha-hacker/test"
device = "cuda"
# try device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [672]:
!rm submission.csv

In [673]:
transform_1= transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [674]:
transform_2= transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [675]:
transform_3 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((384,288)),
    #384*288
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [676]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 32))
#         img = np.mean(img, axis=2)
#         print(img.shape)
        img = transform_1(img)
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
            return torch.FloatTensor(img / 256), int(label)

    def __len__(self):
        return len(self.data)

In [677]:
class Task2Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task2")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        #img = io.read_image(f"{self.root}/{filename}")
        #img = cv2.resize(img, (224, 224))
        img = transform_2(img)
        if self.return_filename:
            return torch.FloatTensor((img ) / 255), filename
        else:
            return torch.FloatTensor((img ) / 255), label

    def __len__(self):
        return len(self.data)

In [678]:
class Task3Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task3")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        #img = io.read_image(f"{self.root}/{filename}")
        #img = cv2.resize(img, (224, 224))
        img = transform_3(img)
        if self.return_filename:
            return torch.FloatTensor((img ) / 255), filename
        else:
            return torch.FloatTensor((img ) / 255), label

    def __len__(self):
        return len(self.data)

In [679]:
class ResNet18_1(nn.Module):
    def __init__(self):
        super(ResNet18_1, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 10)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [680]:
class ResNet18_2(nn.Module):
    def __init__(self):
        super(ResNet18_2, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 72)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [681]:
class ResNet18_3(nn.Module):
    def __init__(self):
        super(ResNet18_3, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 144)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [682]:
train_data = []
val_data = []

file = open(f'{TRAIN_PATH}/annotations.csv', newline='')
with file as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if random.random() < 0.7:
            train_data.append(row)
        else:
            val_data.append(row)
            
train_ds = Task1Dataset(train_data, root=TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=4, drop_last=True, shuffle=True)

val_ds = Task1Dataset(val_data, root=TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [684]:

test_data = []
PATH_1 = "model_1.pt"
with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

test_ds = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

if os.path.exists('submission.csv'):
    file = open('submission.csv', 'w', newline='')
    csv_writer = csv.writer(file)
else:
    file = open('submission.csv', 'w', newline='')
    csv_writer = csv.writer(file)
    csv_writer.writerow(["filename", "label"])
    
model = ResNet18_1().to(device)
model.load_state_dict(torch.load(PATH_1))
cnt = 0
model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    #print(image.shape)
    #image = image.permute(0,3,1,2)
    
    pred = model(image)
    pred = torch.argmax(pred, dim=1)

    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], str(pred[i].item())])
        cnt += 1
# print(cnt) # 6500

print(cnt)
#file.close()


6500


In [686]:

test_ds = Task2Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=2, drop_last=False, shuffle=False)
PATH_2 = "model_2.pt"
model = ResNet18_2().to(device)
model.load_state_dict(torch.load(PATH_2))

model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    pred = model(image)
    pred = pred.data.cpu().numpy()
    tt=list()
    for j in range(pred.shape[0]):
        ans=""
        temp = np.zeros(72)
        max=-10000
        index=0
        for i in range(36):
            if pred[j][i] > max:
                index=i
                max=pred[j][i]
        #print(index)
        if index >=10 :
            ans+=(str(chr(index+87)))
        else:
            ans+=(str(index))

        max=-10000
        index=0
        for i in range(36):
            if pred[j][i+36] > max:
                index=i
                max=pred[j][i+36]
        if index >=10 :
            ans+=(str(chr(index+87)))
        else:
            ans+=(str(index))
        #print(ans)
        #print('\n')
        tt.append(ans)

    #print(tt)
    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], tt[i]])
        

In [688]:

test_ds = Task3Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=2, drop_last=False, shuffle=False)
PATH_3 = "model_3.pt"
model = ResNet18_3().to(device)
model.load_state_dict(torch.load(PATH_3))

model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    pred = model(image)
    pred = pred.data.cpu().numpy()
    tt=list()
    for j in range(pred.shape[0]):
        ans=""
        max=-10000
        index=0
        for i in range(36):
            if pred[j][i] > max:
                index=i
                max=pred[j][i]
        #print(index)
        if index >=10 :
            ans+=(str(chr(index+87)))
        else:
            ans+=(str(index))

        max=-10000
        index=0
        for i in range(36):
            if pred[j][i+36] > max:
                index=i
                max=pred[j][i+36]
        if index >=10 :
            ans+=(str(chr(index+87)))
        else:
            ans+=(str(index))
            
        max=-10000
        index=0
        for i in range(36):
            if pred[j][i+72] > max:
                index=i
                max=pred[j][i+36]
        if index >=10 :
            ans+=(str(chr(index+87)))
        else:
            ans+=(str(index))
            
        max=-10000
        index=0
        for i in range(36):
            if pred[j][i+108] > max:
                index=i
                max=pred[j][i+36]
        if index >=10 :
            ans+=(str(chr(index+87)))
        else:
            ans+=(str(index))
        #print(ans)
        #print('\n')
        tt.append(ans)

    #print(tt)
    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], tt[i]])
        
file.close()